# Generating and Fitting Simulated Datasets with Bird-Snack

The goal of this notebook is to introduce dataset simulation, and fitting, using Bird-Snack pipelines in the `sbc/` directory.

BirdSnack Simulation-based calibration works by simulating datasets using a user-defined set of input hyperparameters, then fitting simulated datasets, and assessing recovery of input hyperparameters.

The methods for posterior predictive checks are the same, but the input hyperparameters are taken from posterior samples in the birdsnack rootpath directory, i.e. `~/products/stan_fits/FITS/`.

There are various custom plotting scripts also in `sbc/plotting_scripts/`.

Let's start off with vanilla SBC.

---
---
---
# Simulation-Based Calibration

Full script in `perform_sbc.py`

## 1) Set path to sbc model_files, load up classes and `.yaml` choices

In [1]:
#Import classes
import sys, yaml
sys.path.append('sbc/model_files/')
from SBC import *

#User defined choices for simulating datasets
with open('sbc/sbc.yaml') as f:
    sbc_choices = yaml.load(f, Loader=yaml.FullLoader)

#Set paths to sbc/ rootpath directory, and birdsnack root directory
sbc_choices['load_parameters']['path_to_rootpath'] = 'sbc/'
sbc_choices['load_parameters']['path_to_birdsnack_rootpath'] = './'
sbc_choices

{'load_parameters': {'path_to_rootpath': 'sbc/',
  'path_to_birdsnack_rootpath': './'},
 'simulate_parameters': {'Nsims': 200,
  'S': 100,
  'simulator': 'BayeSN',
  'bayesn_parameters': {'bymodel': 'M20', 'thetamode': 'random'},
  'epsmode': 'random',
  'FPC0m': 'bymodel',
  'flts': ['B_CSP', 'V_CSP', 'r_CSP', 'i_CSP', 'J_RC1', 'H_RC'],
  'tauA': 0.5,
  'muRV': 2.5,
  'sigRV': 0.5,
  'AVsimdist': 'Exp',
  'RVsimdist': 'Norm',
  'nuA': 'None',
  'PredefinedIntrinsicHyps': False,
  'PredefinedExtrinsicHyps': False,
  'pre_defined_hyps': {'load_file': 'Fiducial'}},
 'fit_parameters': {'birdsnack_yaml': 'birdsnack_sbc'}}

## 2) Make custom choices for simulating, create edit_dictionary, and call `SBC` class instance

In [2]:
#Set tauA=0.3, muRV=2.5, sigRV=0.1
dust_hyps = dict(zip(['tauA','muRV','sigRV'],[0.3,2.5,0.1]))

#BayeSN installation required for BayeSN simulations, so let's use 'perfect' forward model to simulate datasets,
#i.e. same BirdSnack model used for simulating as fitting
#Also set FPC0m to be a std_normal draw
#Finally, simulate 100 datasets
simulator = {'simulator':'perfect','FPC0m':'random','Nsims':100}

#Create edit dictionary to update values
edit_dict   = {'simulate_parameters':{**dust_hyps,**simulator}}

#Get SBC_CLASS
sbc = SBC_CLASS(sbc_choices,edit_dict)

## 3) Simulate some datasets

In [3]:
sbc.simulate_truths()

Done
########################################


## 4) Fit datasets [commented out to save on time complexity]

In [4]:
#To get stan model running on jupyter notebook
import nest_asyncio
nest_asyncio.apply()

#Fit truths (commented out to save time complexity)
#sbc.fit_truths()

## 5) Plot Results

From here, check out `plot_sbc.py` to see how results can be plotted up.

See also the full `perform_sbc.py` script.

---
---
---

# Posterior Predictive Checks

The other way datasets can be simulated/fitted is to simulate using the posterior median hyperparameters from a BirdSnack fit to real data. Here is a quick demo on how this is done. Full script is `perform_ppc.py`

## 1) Set up paths, load up classes and `.yaml` choices

In [5]:
import sys, yaml
sys.path.append('sbc/model_files/')
from SBC import *
from sbc_plot_functions import update_edit_dict_for_ppc

#This time load up ppc.yaml
with open('sbc/ppc.yaml') as f:
        sbc_choices = yaml.load(f, Loader=yaml.FullLoader)

#Set paths to sbc/ rootpath directory, and birdsnack root directory
sbc_choices['load_parameters']['path_to_rootpath'] = 'sbc/'
sbc_choices['load_parameters']['path_to_birdsnack_rootpath'] = './'
print ("Notice entries here are 'ppc'; these are eventually filled with values from real-data fit \n"+"###"*10)
sbc_choices

Notice entries here are 'ppc'; these are eventually filled with values from real-data fit 
##############################


{'load_parameters': {'path_to_rootpath': 'sbc/',
  'path_to_birdsnack_rootpath': './'},
 'simulate_parameters': {'Nsims': 100,
  'S': 'ppc',
  'tauA': 'ppc',
  'muRV': 'ppc',
  'sigRV': 'ppc',
  'epsmode': 'ppc',
  'FPC0m': 'ppc',
  'simulator': 'perfect',
  'flts': ['B_CSP', 'V_CSP', 'r_CSP', 'i_CSP', 'J_RC1', 'H_RC'],
  'AVsimdist': 'ppc',
  'nuA': 'ppc',
  'RVsimdist': 'ppc',
  'PredefinedIntrinsicHyps': True,
  'PredefinedExtrinsicHyps': True,
  'pre_defined_hyps': {'load_file': 'None',
   'ppc_zero_index': 'None',
   'ppc_IntrinsicModel': 'None'}},
 'fit_parameters': {'birdsnack_yaml': 'birdsnack_sbc'}}

## 2) Use the `update_edit_dict_for_ppc` function to replace 'ppc' entries with real values

In [6]:
#Choices for simulating data based on previous stan fit with BirdSnack
edit_dict = {'simulate_parameters':{'Nsims':100,'pre_defined_hyps':{'load_file':'AVExp_BVcut1.0'}}}

edit_dict = update_edit_dict_for_ppc(sbc_choices,edit_dict)
edit_dict

{'simulate_parameters': {'Nsims': 100,
  'pre_defined_hyps': {'load_file': 'AVExp_BVcut1.0',
   'ppc_zero_index': 1,
   'ppc_IntrinsicModel': 'Deviations'},
  'S': 65,
  'AVsimdist': 'Exp',
  'RVsimdist': 'Norm',
  'tauA': 0.3191,
  'muRV': 2.6133,
  'sigRV': 0.6385}}

## 3) As before, call `SBC` class and simulate datasets

In [7]:
#Get SBC_CLASS
sbc = SBC_CLASS(sbc_choices,edit_dict)

#Simulate SNe Datasets
sbc.simulate_truths()

Done
########################################


## 4) Update the BirdSnack fitting options, e.g. to fit Exp. $A_V^s$ simulated datasets using a Gamma distribution

In [8]:
#PPC Recovery of nuA=1; Applying Gamma dist. to fit Exp-simulated Data
BIRDSNACK_EDIT_DICT = {'analysis_parameters':
                        {'HBM_savekey':'PPC_ExpFitGamma',
                        'AVprior':'Gamma','n_warmup':1000,'n_sampling':3000,'n_thin':1000}}

sbc.fit_truths(edit_dict=BIRDSNACK_EDIT_DICT)

Completed fitting simulated datasets


## 5) Plot results.

See custom plotting scripts in `sbc/plotting_scripts/`.